In [1]:
from dataclasses import dataclass

import time

import pyvisa
from devices import DigitalController, Nanopositioner, MicrowaveGenerator
import TimeTagger

import numpy as np
from scipy.optimize import curve_fit

import plotly.graph_objects as go

In [2]:
SAMPLE_RATE = 1e9
MAX = 32767

rm = pyvisa.ResourceManager()
awg = rm.open_resource("USB0::2391::22279::MY59001893::0::INSTR")

awg.write("*CLS")
awg.write("*RST")

awg.write('FORMat:BORDer SWAP')

awg.write('TRIG1:SOUR EXT')

awg.write('SOUR1:FUNC ARB')
awg.write(f'SOUR1:FUNC:ARB:SRAT {SAMPLE_RATE:.0E}')

27

In [3]:
digital_controller = DigitalController()
nanopositioner = Nanopositioner()
time_tagger = TimeTagger.createTimeTagger()
microwave_generator = MicrowaveGenerator(address="TCPIP0::169.254.209.144::inst0::INSTR")

In [4]:
center = np.array([5.98464424, 6.12179119, 6.46106908])

In [5]:
def gaussian_1d(x, x0, sx, A, offset):
    return A * np.exp(-((x-x0)/sx)**2) + offset

In [6]:
def lorentzian(f, f0, A, Gamma):
    return -A*((0.5*Gamma)/((f-f0)**2 + (0.5*Gamma)**2)) + 1

In [7]:
def take_pixel(binwidth=0.1, n_values=2):
    point_duration = binwidth * n_values 

    counter = TimeTagger.Counter(tagger=time_tagger, channels=[1], binwidth=int(binwidth*1e12), n_values=n_values)
    time.sleep(point_duration)
    return counter.getData()[0, 1:] / binwidth

In [8]:
def find_focus(radius=0.2, resolution = 0.005, disp=True):
    Z = np.arange(center[2]-radius, center[2]+radius+resolution, resolution)

    I = np.zeros(Z.shape)

    for i in range(2):
        nanopositioner.set_position_component(i, center[i])
    for i, z in enumerate(Z):
        nanopositioner.set_position_component(2, z)
        I[i] = np.mean(take_pixel(n_values=5))

    fit, _ = curve_fit(gaussian_1d, Z, I, p0=(center[2], 0.01, 1000, 0))

    if disp:
        figure = go.Figure(data=[go.Scatter(x=Z, y=I),
                                    go.Scatter(x=Z, y=gaussian_1d(Z, *fit))
                                ])
        figure.update_layout(
            width=750,
            height=750,
        )
        figure.show()

    return fit[0], np.max(I)

In [9]:
def take_image(radius, resolution, binwidth=0.1, n_values=2):
        point_duration = binwidth * n_values 

        X = np.arange(center[0]-radius, center[0]+radius+resolution, resolution)
        Y = np.arange(center[1]-radius, center[1]+radius+resolution, resolution)
        I = np.zeros((Y.size, X.size))

        print(f"\nExperiment Duration: {(I.size * point_duration)/60:.2f} minutes.")

        nanopositioner.set_position_component(2, center[2])
        for i, y in enumerate(Y):
            nanopositioner.set_position_component(1, y)
            for j, x in enumerate(X):
                nanopositioner.set_position_component(0, x)
                data = take_pixel(binwidth=binwidth, n_values=n_values)
                I[i, j] = np.mean(data)

        return X, Y, I

In [10]:
def find_center_along_axis(radius, resolution, axis, binwidth=0.1, n_values=2, set_value=True, disp=False):
    R = np.arange(center[axis]-radius[axis], center[axis]+radius[axis]+resolution[axis], resolution[axis])
    I_r = np.zeros(R.size)
    I_r_std = np.zeros(R.size)
    
    if disp:
        point_duration = binwidth * n_values
        print(f"\nExperiment Duration: {(I_r.size * point_duration):.2f} seconds.")

    for i in range(3):
        if i != axis: nanopositioner.set_position_component(i, center[i])
    for i, r in enumerate(R):
        nanopositioner.set_position_component(axis, r)
        data = take_pixel(binwidth, n_values)
        I_r[i] = np.mean(data)
        I_r_std[i] = np.std(data)

    fit, _ = curve_fit(gaussian_1d, R, I_r, p0=(center[axis], 0.01, np.max(I_r), np.min(I_r)))

    if disp:
        print(f'Center: {center[axis]}')
        R_fine = np.linspace(R[0], R[-1], 1000)
        figure = go.Figure(data=[go.Scatter(x=R, y=I_r, error_y=dict(type='data', array=I_r_std, visible=True)),
                                go.Scatter(x=R_fine, y=gaussian_1d(R_fine, *fit))])
        figure.update_layout(
            width=750,
            height=750,
        )
        figure.show()

    if set_value:
        center[axis] = fit[0]

    return fit[0]

def find_center(radius=np.array([0.03, 0.03, 0.09]), resolution=np.array([0.003, 0.003, 0.009]), binwidth=0.1, n_values=2, disp=False, set_value=True):
    center = np.array([ find_center_along_axis(radius, resolution, i, binwidth=binwidth, n_values=n_values, set_value=set_value, disp=disp) for i in (2, 0, 1) ])[np.array([1, 2, 0])]
    return center

In [11]:
def create_counters(n_values = 200000):
    time_tagger.setTriggerLevel(channel=3, voltage=0.40)     
    time_tagger.setTriggerLevel(channel=4, voltage=0.40)

    reference_counter = TimeTagger.CountBetweenMarkers(
        tagger=time_tagger, 
        click_channel=1, 
        begin_channel=digital_controller.reference, 
        end_channel=-digital_controller.reference, 
        n_values=n_values)

    signal_counter = TimeTagger.CountBetweenMarkers(
        tagger=time_tagger, 
        click_channel=1, 
        begin_channel=digital_controller.signal, 
        end_channel=-digital_controller.signal, 
        n_values=n_values)
    
    return reference_counter, signal_counter

In [12]:
def awg_rabi(duration):
    duration /= 2
    samples = np.around(duration).astype(int)
    waveform = np.pad(np.ones(samples).astype(int) * MAX, pad_width=(0, 50))
    
    awg.write('SOUR1:DATA:VOL:CLE')

    awg.write_binary_values('SOUR1:DATA:ARB:DAC PULSE, ', waveform, datatype='i')
    
    sequence_command = f'RABI,PULSE,0,onceWaitTrig,highAtStart,0'
    sequence_length = len(sequence_command.encode('ascii'))
    awg.write(f'SOUR1:DATA:SEQ #{len(str(sequence_length))}{sequence_length}' + sequence_command)

    awg.write('SOUR1:FUNC:ARB RABI')
    awg.write('SOUR1:VOLT 4')

    awg.write('OUTP1 ON')

In [13]:
@dataclass
class Sequence:
    green: np.array
    microwave: np.array
    awg: np.array
    reference: np.array
    signal: np.array

In [14]:
def write_sequence(sequence):
    digital_controller.write_digital(digital_controller.green, sequence.green)
    digital_controller.write_digital(digital_controller.reference, sequence.reference)
    digital_controller.write_digital(digital_controller.microwave, sequence.microwave)
    digital_controller.write_digital(digital_controller.awg, sequence.awg)
    digital_controller.write_digital(digital_controller.signal, sequence.signal)

In [15]:
def create_odmr_sequence():
    point_duration = 5 # microseconds
    sequence_duration = 10  # microseconds

    green_pulse = np.ones(sequence_duration * 100, dtype=int) 

    reference_pulse = np.zeros(sequence_duration * 100, dtype=int) 
    reference_pulse[:point_duration * 100] = 1

    microwave_pulse = np.zeros(sequence_duration * 100, dtype=int)
    microwave_pulse[point_duration * 100:] = 1

    signal_pulse = microwave_pulse

    awg_pulse = np.zeros(sequence_duration)

    return Sequence(green_pulse, microwave_pulse, awg_pulse, reference_pulse, signal_pulse)

In [16]:
def create_rabi_sequence(duration):
    duration = np.ceil(duration / 10).astype(int)

    sequence_duration = 1000
    green_initialization_duration = 203
    green_measurement_duration = 36
    collection_duration = 30

    green_delay_duration = np.array([51, 47, 30, 50])
    green_duration_offset = -6

    padding = 2

    green_pulse = np.zeros(sequence_duration)
    microwave_pulse = np.zeros(sequence_duration)
    awg_pulse = np.zeros(sequence_duration)
    reference_pulse = np.zeros(sequence_duration)
    signal_pulse = np.zeros(sequence_duration)

    current_time = 0

    green_pulse[current_time:current_time+green_initialization_duration] = 1
    reference_pulse[current_time+green_delay_duration[0]:current_time+green_delay_duration[0]+green_initialization_duration+green_duration_offset] = 1
    current_time += green_initialization_duration

    microwave_pulse[current_time+green_delay_duration[1]:current_time+duration+1+green_delay_duration[1]] = 1
    awg_pulse[current_time+green_delay_duration[2]:] = 1

    current_time += duration + padding

    green_pulse[current_time:current_time+green_measurement_duration] = 1
    signal_pulse[current_time+green_delay_duration[3]:current_time+green_delay_duration[3]+collection_duration] = 1

    return Sequence(green_pulse, microwave_pulse, awg_pulse, reference_pulse, signal_pulse)

In [17]:
def rabi(durations, num_points, point_duration=1, 
         microwave_amplitude=0, recentering=True):
    experiment_duration = durations.size * ((13.3 if recentering else 0) + num_points * point_duration)
    print(f"Experiment Duration: {experiment_duration/60:.2f} minutes.\n")

    reference_rates = np.zeros((durations.size, num_points))
    signal_rates = np.zeros((durations.size, num_points))

    digital_controller.reset_digital()

    microwave_generator.set_amplitude(microwave_amplitude)
    microwave_generator.set_frequency(transition_frequency, 'GHz')

    microwave_generator.enable_output()

    for d, duration in enumerate(durations):
        print(f"Duration: {duration}")
        sequence = create_rabi_sequence(duration)
        
        if recentering:
            print("Recentering...")
            find_center(disp=False)
            nanopositioner.set_position(center)

        awg_rabi(duration)

        for i in range(num_points):
            reference_counter, signal_counter = create_counters()

            write_sequence(sequence)
            digital_controller.configure_digital(True)

            time.sleep(point_duration)

            digital_controller.reset_digital()
            digital_controller.configure_digital(True)

            reference_duration = np.sum(reference_counter.getBinWidths()) * 1e-12
            signal_duration = np.sum(signal_counter.getBinWidths()) * 1e-12

            reference_rates[d, i] = np.sum(reference_counter.getData()) / reference_duration
            signal_rates[d, i] = np.sum(signal_counter.getData()) / signal_duration

            print(f"{i:0>2} (R: {reference_rates[d, i]:.0f}, S: {signal_rates[d, i]:.0f})")
        normalized_rates = signal_rates[d]/reference_rates[d]
        normalized_rate_mean = np.mean(normalized_rates)
        normalized_rate_ste = np.std(normalized_rates) / np.sqrt(num_points)
        print(f"N: {normalized_rate_mean:.4f} ± {normalized_rate_ste:.4f}\n")

    microwave_generator.disable_output()

    normalized_rates = signal_rates / reference_rates

    normalized_rate_mean = np.mean(normalized_rates, axis=1)
    normalized_rate_ste = np.std(normalized_rates, axis=1) / np.sqrt(num_points)

    return normalized_rate_mean, normalized_rate_ste

In [18]:
digital_controller.reset_digital()
digital_controller.enable_digital(digital_controller.green)

In [19]:
z, I = find_focus()

In [20]:
center

array([5.98464424, 6.12179119, 6.46106908])

In [21]:
center[2] = z
center

array([5.98464424, 6.12179119, 6.4646614 ])

In [24]:
radius = 0.03
resolution = radius/10

X, Y, I = take_image(radius, resolution)

figure = go.Figure(data=[go.Heatmap(x=X, y=Y,z=I)])
figure.update_layout(
    width=750,
    height=750,
)
figure.show()


Experiment Duration: 1.61 minutes.


In [25]:
with open('data/confocal_microscopy_narrow_3_9.npy', 'wb') as f:
    np.save(f, X)
    np.save(f, Y)
    np.save(f, I)

In [341]:
radius = 0.1
resolution = 0.005

X, Y, I = take_image(radius, resolution)

figure = go.Figure(data=[go.Heatmap(x=X, y=Y,z=I)])
figure.update_layout(
    width=750,
    height=750,
)
figure.show()


Experiment Duration: 5.60 minutes.


In [21]:
center = np.array([5.98974606, 6.12680757, 6.48432126])

In [22]:
nanopositioner.set_position(center)

In [62]:
find_center(disp=True)
center


Experiment Duration: 4.40 seconds.
Center: 6.463652989067042



Experiment Duration: 4.40 seconds.
Center: 5.979850178565545



Experiment Duration: 4.40 seconds.
Center: 6.121132945101026


array([5.97853806, 6.12057007, 6.45469859])

In [33]:
def odmr(frequency_center=2.8, frequency_radius=0.05, frequency_resolution=0.005, point_duration=0.5, num_points=5,
         recentering=False):
    frequencies = np.arange(frequency_center - frequency_radius, frequency_center + frequency_radius + frequency_resolution, frequency_resolution)

    experiment_duration = frequencies.size * ((13.2 if recentering else 0) + num_points * point_duration)
    print(f"Experiment Duration: {experiment_duration/60:.2f} minutes.\n")

    reference_rates = np.zeros((frequencies.size, num_points))
    signal_rates = np.zeros((frequencies.size, num_points))

    digital_controller.reset_digital()

    nanopositioner.set_position(center)

    microwave_generator.set_amplitude(0)
    microwave_generator.enable_output()

    for f, frequency in enumerate(frequencies):
        print(f"Frequency: {frequency:.3f}")
        microwave_generator.set_frequency(frequency, 'GHz')

        if recentering:
            print("Recentering...")
            find_center(disp=False)
            nanopositioner.set_position(center)

        for i in range(num_points):
            reference_counter, signal_counter = create_counters()

            write_sequence(create_odmr_sequence())
            digital_controller.configure_digital(True)

            time.sleep(point_duration)

            digital_controller.reset_digital()

            reference_duration = np.sum(reference_counter.getBinWidths()) * 1e-12
            signal_duration = np.sum(signal_counter.getBinWidths()) * 1e-12

            reference_rates[f, i] = np.sum(reference_counter.getData()) / reference_duration
            signal_rates[f, i] = np.sum(signal_counter.getData()) / signal_duration

            print(f"{i:0>2} (R: {reference_rates[f, i]:.0f}, S: {signal_rates[f, i]:.0f})")

        normalized_rates = signal_rates[f]/reference_rates[f]
        normalized_rate_mean = np.mean(normalized_rates)
        normalized_rate_ste = np.std(normalized_rates) / np.sqrt(num_points)
        print(f"N: {normalized_rate_mean:.4f} ± {normalized_rate_ste:.4f}\n")

    microwave_generator.disable_output()

    normalized_rates = signal_rates / reference_rates

    normalized_rate_mean = np.mean(normalized_rates, axis=1)
    normalized_rate_ste = np.std(normalized_rates, axis=1) / np.sqrt(num_points) 

    return frequencies, normalized_rate_mean, normalized_rate_ste

In [28]:
microwave_generator.disable_modulation()

In [34]:
frequencies, normalized_rate_mean, normalized_rate_ste = odmr(frequency_center=2.80, frequency_radius=0.05)

Experiment Duration: 0.88 minutes.

Frequency: 2.750
00 (R: 31079, S: 30266)
01 (R: 30323, S: 30064)
02 (R: 29646, S: 29745)
03 (R: 30020, S: 29416)
04 (R: 29582, S: 28687)
N: 0.9836 ± 0.0055

Frequency: 2.755
00 (R: 30009, S: 29509)
01 (R: 29358, S: 28586)
02 (R: 29232, S: 29132)
03 (R: 28996, S: 28493)
04 (R: 29874, S: 28412)
N: 0.9775 ± 0.0067

Frequency: 2.760
00 (R: 29323, S: 28465)
01 (R: 28949, S: 28057)
02 (R: 28254, S: 27889)
03 (R: 28747, S: 28329)
04 (R: 28813, S: 27989)
N: 0.9768 ± 0.0035

Frequency: 2.765
00 (R: 28398, S: 27346)
01 (R: 28255, S: 27436)
02 (R: 29010, S: 28527)
03 (R: 29595, S: 28419)
04 (R: 27079, S: 27080)
N: 0.9755 ± 0.0065

Frequency: 2.770
00 (R: 27206, S: 26596)
01 (R: 27066, S: 26438)
02 (R: 27294, S: 26063)
03 (R: 27760, S: 26645)
04 (R: 27606, S: 26008)
N: 0.9623 ± 0.0060

Frequency: 2.775
00 (R: 27098, S: 25920)
01 (R: 27038, S: 26103)
02 (R: 27630, S: 25628)
03 (R: 27206, S: 24970)
04 (R: 27213, S: 25512)
N: 0.9410 ± 0.0079

Frequency: 2.780
00 (R

In [35]:
fit, err = curve_fit(lorentzian, frequencies, normalized_rate_mean, p0=(2.65, 0.01, 0.05))

frequencies_fine = np.linspace(frequencies[0], frequencies[-1], 1000)

figure = go.Figure(data=[
    go.Scatter(x=frequencies, y=normalized_rate_mean, error_y=dict(type='data', array=normalized_rate_ste, visible=True), name="Signal"),
    go.Scatter(x=frequencies_fine, y=lorentzian(frequencies_fine, *fit), name="Fit"),
])

figure.update_layout(
    width=750,
    height=750,
    title='ODMR Spectrum',
    xaxis=dict(title='Frequency (GHz)'),
    yaxis=dict(title='Normalized PL Counts (Arb.)')
)
figure.show()

In [36]:
with open('data/odmr_3_9.npy', 'wb') as f:
    np.save(f, frequencies)
    np.save(f, normalized_rate_mean)
    np.save(f, normalized_rate_ste)

In [38]:
transition_frequency = fit[0]

In [39]:
microwave_generator.enable_modulation()

In [40]:
microwave_generator.disable_output()

In [41]:
durations = np.arange(0, 200+10, 5) + 2
num_points = 13
microwave_amplitude = 0

normalized_rate_mean, normalized_rate_ste = rabi(durations, num_points, microwave_amplitude=microwave_amplitude)

Experiment Duration: 18.41 minutes.

Duration: 2
Recentering...
00 (R: 32484, S: 31659)
01 (R: 32706, S: 32497)
02 (R: 32737, S: 32768)
03 (R: 32852, S: 33082)
04 (R: 32334, S: 30915)
05 (R: 32249, S: 31550)
06 (R: 31881, S: 32291)
07 (R: 31806, S: 31315)
08 (R: 31265, S: 31146)
09 (R: 32276, S: 31004)
10 (R: 31206, S: 32015)
11 (R: 31673, S: 32856)
12 (R: 32765, S: 32203)
N: 0.9931 ± 0.0064

Duration: 7
Recentering...
00 (R: 31240, S: 29878)
01 (R: 30728, S: 28737)
02 (R: 31407, S: 30538)
03 (R: 31940, S: 29702)
04 (R: 31091, S: 28606)
05 (R: 30824, S: 30824)
06 (R: 30790, S: 28429)
07 (R: 30614, S: 29281)
08 (R: 31325, S: 30139)
09 (R: 31463, S: 29571)
10 (R: 31215, S: 31462)
11 (R: 30467, S: 30341)
12 (R: 30864, S: 31517)
N: 0.9631 ± 0.0090

Duration: 12
Recentering...
00 (R: 31277, S: 29001)
01 (R: 31146, S: 30723)
02 (R: 30113, S: 29154)
03 (R: 30781, S: 27649)
04 (R: 30804, S: 26852)
05 (R: 31333, S: 27414)
06 (R: 31933, S: 30335)
07 (R: 30835, S: 28336)
08 (R: 30076, S: 29711)
0

In [42]:
figure = go.Figure(data=[
    go.Scatter(x=durations, y=normalized_rate_mean, 
               error_y=dict(type='data', array=normalized_rate_ste, visible=True), 
               name="Signal"),
])

figure.update_layout(
    width=750,
    height=750,
    title='Rabi Experiment',
    xaxis=dict(title='Durations (ns)'),
    yaxis=dict(title='Normalized PL Counts (Arb.)')
)
figure.show()

In [46]:
with open('data/rabi_3_9.npy', 'wb') as f:
    np.save(f, durations)
    np.save(f, normalized_rate_mean)
    np.save(f, normalized_rate_ste)

In [43]:
def damped_oscillator(time, decay_time, amplitude, period, phase, offset):
    return amplitude * np.exp(-time/decay_time)*(np.cos((2*np.pi/period)*time-phase)) + offset

In [47]:
def find_pi_duration(durations, normalized_rate_mean, disp=False):
    fit, err = curve_fit(damped_oscillator, durations, normalized_rate_mean, 
                     p0=(1000, 0.1, 70, 0, 0.85),
                     bounds=([0, 0, 0, -np.pi, 0], [1000, 1, 500, np.pi, 1])
                     )

    decay_time = fit[0]
    decay_time_uncertainty = np.sqrt(err[0, 0])
    print(f"Decay Time: {decay_time:.2f} ± {decay_time_uncertainty:.2f} nanoseconds.")

    period = fit[2]
    period_uncertainty = np.sqrt(err[2, 2])
    print(f"Period: {period:.2f} ± {period_uncertainty:.3f} nanoseconds.")

    phase = fit[3]
    phase_uncertainty = np.sqrt(err[3, 3])
    print(f"Phase: {phase:.2f} ± {phase_uncertainty:.4f} radians.")

    print()

    pi_2_duration = (phase + (np.pi/2))/(2*np.pi/period)
    pi_2_duration_uncertainty = pi_2_duration * np.sqrt((period_uncertainty/period)**2 + (phase_uncertainty/(phase + (np.pi/2)))**2)
    print(f"Pi/2 Duration: {pi_2_duration:.2f} ± {pi_2_duration_uncertainty:.4f} nanoseconds.")

    pi_duration = (phase + (np.pi))/(2*np.pi/period)
    pi_duration_uncertainty = pi_duration * np.sqrt((period_uncertainty/period)**2 + (phase_uncertainty/(phase + np.pi))**2)
    print(f"Pi Duration: {pi_duration:.2f} ± {pi_duration_uncertainty:.4f} nanoseconds.")

    if disp:
        finer_durations = np.linspace(durations[0], durations[-1], 1000)

        figure = go.Figure(data=[
            go.Scatter(x=durations, y=normalized_rate_mean, mode='markers', name="Data"),
            go.Scatter(x=finer_durations, y=damped_oscillator(finer_durations, *fit), name="Fit")]
            
        )

        figure.update_layout(
            title='Rabi Experiment',
            xaxis=dict(title='Durations (ns)'),
            yaxis=dict(title='Normalized PL Counts (Arb.)')
        )
        figure.show()

    return pi_2_duration, pi_duration

In [48]:
pi_2_duration, pi_duration = find_pi_duration(durations, normalized_rate_mean, disp=True)

Decay Time: 698.87 ± 305.25 nanoseconds.
Period: 69.09 ± 0.518 nanoseconds.
Phase: 0.31 ± 0.0767 radians.

Pi/2 Duration: 20.64 ± 0.8578 nanoseconds.
Pi Duration: 37.92 ± 0.8903 nanoseconds.


In [49]:
pi_2_duration_rounded = np.round(pi_2_duration).astype(int)
pi_duration_rounded = np.round(pi_duration).astype(int)

pi_2_duration_rounded, pi_duration_rounded

(21, 38)

In [50]:
def awg_ramsey(duration):
    awg.write("SOUR1:DATA:VOL:CLE")

    waveform = np.zeros(pi_2_duration_rounded + duration//2).astype(int)
    current_time = 0

    waveform[current_time:current_time+pi_2_duration_rounded//2] = 1
    current_time += pi_2_duration_rounded//2

    waveform[current_time:current_time+duration//2] = 0
    current_time += duration//2

    waveform[current_time:current_time+pi_2_duration_rounded//2] = 1
    current_time += pi_2_duration_rounded//2

    awg.write_binary_values(
            f'SOUR1:DATA:ARB:DAC PULSE, ', 
            waveform * MAX, 
            datatype='i')
    
    sequence_command = f'RAMSEY,PULSE,0,onceWaitTrig,highAtStart,0'
    sequence_length = len(sequence_command.encode('ascii'))
    awg.write(f'SOUR1:DATA:SEQ #{len(str(sequence_length))}{sequence_length}' + sequence_command)

    awg.write(f'SOUR1:FUNC:ARB RAMSEY')
    awg.write(f'SOUR1:VOLT 4')

    awg.write(f'OUTP1 ON')

In [51]:
find_center()

array([5.9823431 , 6.12345753, 6.48446785])

In [52]:
def create_ramsey_sequence(duration):
    duration = np.ceil(duration / 10).astype(int)
    
    pi_2_duration_rounded_samples = np.ceil(pi_2_duration_rounded / 10).astype(int)

    sequence_duration = 1000
    green_initialization_duration = 203
    green_measurement_duration = 36
    collection_duration = 30

    green_delay_duration = np.array([51, 47, 30, 50])
    green_duration_offset = -6

    padding = 2

    green_pulse = np.zeros(sequence_duration)
    microwave_pulse = np.zeros(sequence_duration)
    awg_pulse = np.zeros(sequence_duration)
    reference_pulse = np.zeros(sequence_duration)
    signal_pulse = np.zeros(sequence_duration)

    current_time = 0

    green_pulse[current_time:current_time+green_initialization_duration] = 1
    reference_pulse[current_time+green_delay_duration[0]:current_time+green_delay_duration[0]+green_initialization_duration+green_duration_offset] = 1
    current_time += green_initialization_duration

    microwave_pulse[current_time+green_delay_duration[1]:current_time+2*pi_2_duration_rounded_samples+duration+7+1+green_delay_duration[1]] = 1
    awg_pulse[current_time+green_delay_duration[2]:] = 1
    current_time += 2*pi_2_duration_rounded_samples+duration+7

    green_pulse[current_time:current_time+green_measurement_duration] = 1
    signal_pulse[current_time+green_delay_duration[3]:current_time+green_delay_duration[3]+collection_duration] = 1

    return Sequence(green_pulse, microwave_pulse, awg_pulse, reference_pulse, signal_pulse)

In [185]:
duration = 500
awg_ramsey(duration)
write_sequence(create_ramsey_sequence(duration))
digital_controller.configure_digital(True)

In [53]:
def ramsey(durations, num_points, point_duration=1, 
           microwave_amplitude=0, recentering=True):
    experiment_duration = durations.size * ((13.3 if recentering else 0) + num_points * point_duration)
    print(f"Experiment Duration: {experiment_duration/60:.2f} minutes.\n")

    reference_rates = np.zeros((durations.size, num_points))
    signal_rates = np.zeros((durations.size, num_points))

    digital_controller.reset_digital()

    microwave_generator.set_amplitude(microwave_amplitude)
    microwave_generator.set_frequency(transition_frequency, 'GHz')

    microwave_generator.enable_output()

    for d, duration in enumerate(durations):
        print(f"Duration: {duration}")
        sequence = create_ramsey_sequence(duration)
        
        if recentering:
            print("Recentering...")
            find_center(disp=False)
            nanopositioner.set_position(center)

        awg_ramsey(duration)

        for i in range(num_points):
            reference_counter, signal_counter = create_counters()

            write_sequence(sequence)
            digital_controller.configure_digital(True)

            time.sleep(point_duration)

            digital_controller.reset_digital()
            digital_controller.configure_digital(True)

            reference_duration = np.sum(reference_counter.getBinWidths()) * 1e-12
            signal_duration = np.sum(signal_counter.getBinWidths()) * 1e-12

            reference_rates[d, i] = np.sum(reference_counter.getData()) / reference_duration
            signal_rates[d, i] = np.sum(signal_counter.getData()) / signal_duration

            print(f"{i:0>2} (R: {reference_rates[d, i]:.0f}, S: {signal_rates[d, i]:.0f})")
        normalized_rates = signal_rates[d]/reference_rates[d]
        normalized_rate_mean = np.mean(normalized_rates)
        normalized_rate_ste = np.std(normalized_rates) / np.sqrt(num_points)
        print(f"N: {normalized_rate_mean:.4f} ± {normalized_rate_ste:.4f}\n")

    microwave_generator.disable_output()

    normalized_rates = signal_rates / reference_rates

    normalized_rate_mean = np.mean(normalized_rates, axis=1)
    normalized_rate_ste = np.std(normalized_rates, axis=1) / np.sqrt(num_points)

    return normalized_rate_mean, normalized_rate_ste

In [186]:
digital_controller.reset_digital()

In [ ]:
find_center(disp=True)

In [181]:
microwave_generator.disable_output()

In [67]:
durations = np.arange(0, 2000+50, 50)
num_points = 13
microwave_amplitude = 0

normalized_rate_mean, normalized_rate_ste = ramsey(durations, num_points, microwave_amplitude=microwave_amplitude)

Experiment Duration: 17.97 minutes.

Duration: 0
Recentering...
00 (R: 30510, S: 24488)
01 (R: 30489, S: 21584)
02 (R: 31224, S: 22040)
03 (R: 31581, S: 23979)
04 (R: 32563, S: 22940)
05 (R: 31339, S: 24282)
06 (R: 31626, S: 22481)
07 (R: 31068, S: 22681)
08 (R: 30176, S: 23231)
09 (R: 30850, S: 23242)
10 (R: 29601, S: 22932)
11 (R: 30332, S: 20922)
12 (R: 29812, S: 23516)
N: 0.7440 ± 0.0100

Duration: 50
Recentering...
00 (R: 30577, S: 24916)
01 (R: 29684, S: 25287)
02 (R: 30090, S: 25485)
03 (R: 31020, S: 25304)
04 (R: 29502, S: 25240)
05 (R: 29849, S: 26084)
06 (R: 30011, S: 25511)
07 (R: 29564, S: 26628)
08 (R: 29364, S: 24577)
09 (R: 30422, S: 26143)
10 (R: 29705, S: 24875)
11 (R: 30401, S: 25223)
12 (R: 31671, S: 24384)
N: 0.8418 ± 0.0084

Duration: 100
Recentering...
00 (R: 32035, S: 26338)
01 (R: 31540, S: 25707)
02 (R: 30317, S: 26868)
03 (R: 30662, S: 25438)
04 (R: 31044, S: 25042)
05 (R: 29475, S: 28262)
06 (R: 31202, S: 27151)
07 (R: 30521, S: 26781)
08 (R: 30691, S: 25955)

In [58]:
with open('data/ramsey_3_9.npy', 'wb') as f:
    np.save(f, durations)
    np.save(f, normalized_rate_mean)
    np.save(f, normalized_rate_ste)

In [56]:
fit, err = curve_fit(damped_oscillator, durations, normalized_rate_mean, 
                     p0=(100, 1, 300, 0, 0.92),
                     bounds=([0, 0, 0, -np.pi, 0], [1000, 1, 1000, np.pi, 1])
                     )

decay_time = fit[0]
decay_time_uncertainty = np.sqrt(err[0, 0])
print(f"Decay Time: {decay_time *10:.2f} ± {decay_time_uncertainty *10:.2f} nanoseconds.")

finer_durations = np.linspace(durations[0], durations[-1], 1000)

figure = go.Figure(data=[
    go.Scatter(x=durations, y=normalized_rate_mean, error_y=dict(type='data', array=normalized_rate_ste, visible=True), name="Signal"),
    go.Scatter(x=finer_durations, y=damped_oscillator(finer_durations, *fit)),
])

figure.update_layout(
    width=750,
    height=750,
    title='Ramsey Experiment',
    xaxis=dict(title='Durations (ns)'),
    yaxis=dict(title='Normalized PL Counts (Arb.)')
)
figure.show()

Decay Time: 4129.14 ± 720.74 nanoseconds.


In [59]:
def awg_hahn(duration):
    awg.write("SOUR1:DATA:VOL:CLE")

    waveform = np.zeros(pi_2_duration_rounded + duration + pi_duration_rounded//2).astype(int)
    current_time = 0

    waveform[current_time:current_time+pi_2_duration_rounded//2] = 1
    current_time += pi_2_duration_rounded//2

    waveform[current_time:current_time+duration//2] = 0
    current_time += duration//2

    waveform[current_time:current_time+pi_duration_rounded//2] = 1
    current_time += pi_duration_rounded//2

    waveform[current_time:current_time+duration//2] = 0
    current_time += duration//2

    waveform[current_time:current_time+pi_2_duration_rounded//2] = 1
    current_time += pi_2_duration_rounded//2

    awg.write_binary_values(
            f'SOUR1:DATA:ARB:DAC PULSE, ', 
            waveform * MAX, 
            datatype='i')
    
    sequence_command = f'HAHN,PULSE,0,onceWaitTrig,highAtStart,0'
    sequence_length = len(sequence_command.encode('ascii'))
    awg.write(f'SOUR1:DATA:SEQ #{len(str(sequence_length))}{sequence_length}' + sequence_command)

    awg.write(f'SOUR1:FUNC:ARB HAHN')
    awg.write(f'SOUR1:VOLT 4')

    awg.write(f'OUTP1 ON')

In [60]:
def create_hahn_sequence(duration):
    duration = np.ceil(duration / 10).astype(int)
    
    pi_2_duration_rounded_samples = np.ceil(pi_2_duration_rounded / 10).astype(int)
    pi_duration_rounded_samples = np.ceil(pi_duration_rounded / 10).astype(int)

    sequence_duration = 1000
    green_initialization_duration = 203
    green_measurement_duration = 36
    collection_duration = 30

    green_delay_duration = np.array([51, 47, 30, 50])
    green_duration_offset = -6

    padding = 2

    green_pulse = np.zeros(sequence_duration)
    microwave_pulse = np.zeros(sequence_duration)
    awg_pulse = np.zeros(sequence_duration)
    reference_pulse = np.zeros(sequence_duration)
    signal_pulse = np.zeros(sequence_duration)

    current_time = 0

    green_pulse[current_time:current_time+green_initialization_duration] = 1
    reference_pulse[current_time+green_delay_duration[0]:current_time+green_delay_duration[0]+green_initialization_duration+green_duration_offset] = 1
    current_time += green_initialization_duration

    wait_duration = 2*pi_2_duration_rounded_samples+2*duration+pi_duration_rounded_samples+2
    microwave_pulse[current_time+green_delay_duration[1]:current_time+wait_duration+1+green_delay_duration[1]] = 1
    awg_pulse[current_time+green_delay_duration[2]:] = 1
    current_time += wait_duration

    green_pulse[current_time:current_time+green_measurement_duration] = 1
    signal_pulse[current_time+green_delay_duration[3]:current_time+green_delay_duration[3]+collection_duration] = 1

    return Sequence(green_pulse, microwave_pulse, awg_pulse, reference_pulse, signal_pulse)

In [61]:
def hahn(durations, num_points, point_duration=1, 
         microwave_amplitude=0, recentering=True):
    experiment_duration = durations.size * ((13.3 if recentering else 0) + num_points * point_duration)
    print(f"Experiment Duration: {experiment_duration/60:.2f} minutes.\n")

    reference_rates = np.zeros((durations.size, num_points))
    signal_rates = np.zeros((durations.size, num_points))

    digital_controller.reset_digital()

    microwave_generator.set_amplitude(microwave_amplitude)
    microwave_generator.set_frequency(transition_frequency, 'GHz')

    microwave_generator.enable_output()

    for d, duration in enumerate(durations):
        print(f"Duration: {duration}")
        sequence = create_hahn_sequence(duration)
        
        if recentering:
            print("Recentering...")
            find_center(disp=False)
            nanopositioner.set_position(center)

        awg_hahn(duration)

        for i in range(num_points):
            reference_counter, signal_counter = create_counters()

            write_sequence(sequence)
            digital_controller.configure_digital(True)

            time.sleep(point_duration)

            digital_controller.reset_digital()
            digital_controller.configure_digital(True)

            reference_duration = np.sum(reference_counter.getBinWidths()) * 1e-12
            signal_duration = np.sum(signal_counter.getBinWidths()) * 1e-12

            reference_rates[d, i] = np.sum(reference_counter.getData()) / reference_duration
            signal_rates[d, i] = np.sum(signal_counter.getData()) / signal_duration

            print(f"{i:0>2} (R: {reference_rates[d, i]:.0f}, S: {signal_rates[d, i]:.0f})")
        normalized_rates = signal_rates[d]/reference_rates[d]
        normalized_rate_mean = np.mean(normalized_rates)
        normalized_rate_ste = np.std(normalized_rates) / np.sqrt(num_points)
        print(f"N: {normalized_rate_mean:.4f} ± {normalized_rate_ste:.4f}\n")

    microwave_generator.disable_output()

    normalized_rates = signal_rates / reference_rates

    normalized_rate_mean = np.mean(normalized_rates, axis=1)
    normalized_rate_ste = np.std(normalized_rates, axis=1) / np.sqrt(num_points)

    return normalized_rate_mean, normalized_rate_ste

In [223]:
duration = 500
awg_hahn(duration)
write_sequence(create_hahn_sequence(duration))
digital_controller.configure_digital(True)

In [224]:
digital_controller.reset_digital()

In [64]:
durations = np.arange(0, 2000+20, 20)
num_points = 13
microwave_amplitude = 0

normalized_rate_mean, normalized_rate_ste = hahn(durations, num_points, microwave_amplitude=microwave_amplitude)

Experiment Duration: 44.27 minutes.

Duration: 0
Recentering...
00 (R: 31382, S: 30499)
01 (R: 32001, S: 29625)
02 (R: 31178, S: 29994)
03 (R: 30913, S: 29634)
04 (R: 31932, S: 31576)
05 (R: 32004, S: 30342)
06 (R: 32396, S: 30565)
07 (R: 32483, S: 30101)
08 (R: 31908, S: 30594)
09 (R: 30948, S: 29697)
10 (R: 30751, S: 29612)
11 (R: 29792, S: 30253)
12 (R: 30407, S: 30131)
N: 0.9625 ± 0.0067

Duration: 20
Recentering...
00 (R: 31453, S: 24444)
01 (R: 30760, S: 22752)
02 (R: 31752, S: 22871)
03 (R: 31063, S: 23814)
04 (R: 30130, S: 21723)
05 (R: 32164, S: 22082)
06 (R: 31068, S: 22587)
07 (R: 31097, S: 23742)
08 (R: 31576, S: 23152)
09 (R: 32745, S: 23314)
10 (R: 31335, S: 24511)
11 (R: 31050, S: 22342)
12 (R: 31814, S: 24622)
N: 0.7402 ± 0.0079

Duration: 40
Recentering...
00 (R: 31677, S: 24449)
01 (R: 31286, S: 24632)
02 (R: 30680, S: 22778)
03 (R: 31259, S: 24564)
04 (R: 31000, S: 22651)
05 (R: 30715, S: 26150)
06 (R: 31889, S: 25574)
07 (R: 30866, S: 25084)
08 (R: 30952, S: 23313)


In [66]:
with open('data/hahn_3_9.npy', 'wb') as f:
    np.save(f, durations)
    np.save(f, normalized_rate_mean)
    np.save(f, normalized_rate_ste)

In [65]:
figure = go.Figure(data=[
    go.Scatter(x=durations, y=normalized_rate_mean, error_y=dict(type='data', array=normalized_rate_ste, visible=True), name="Signal"),
])

figure.update_layout(
    width=750,
    height=750,
    title='Hahn Experiment',
    xaxis=dict(title='Durations (ns)'),
    yaxis=dict(title='Normalized PL Counts (Arb.)')
)
figure.show()

In [69]:
center

array([5.97704125, 6.11695885, 6.44581344])

In [70]:
transition_frequency

2.799493988118631

In [71]:
pi_duration, pi_2_duration

(37.91576345237031, 20.644499411886073)

In [72]:
pi_duration_rounded, pi_2_duration_rounded

(38, 21)

# Close

In [73]:
del digital_controller
TimeTagger.freeTimeTagger(time_tagger)

True

In [75]:
digital_controller = DigitalController()

In [76]:
digital_controller.disable_digital(digital_controller.green)